# Experimenting With APIs

In this notebook, we will be experimenting with the APIs and create helper functions.

In [1]:
from tqdm.auto import tqdm
import itertools

import os

# my imports
from helpers import kube
from helpers import workload
from helpers import util
from helpers import request_funcs

# experiment utility functionality used in experiments
from exp_util import *

starting watch thread...


## Kubernetes API

We use the kubernetes API to fetch the current status of each deployment and see how many instances they have.

In [2]:
# print details of a deployment
get_knative_watch_info('bench1')

{'kind': 'Deployment',
 'replicas': 0,
 'ready_replicas': 0,
 'last_update': 1614288107.6768322}

## Deploying On Knative

In this section, we will develop the functionality to deploy a given workload to **knative** using the `kn` CLI.
The other options were using [CRDs](https://stackoverflow.com/questions/61384188/how-to-deploy-a-knative-service-with-kubernetes-python-client-library),
or using [kubectl apply](https://developers.redhat.com/coderland/serverless/deploying-serverless-knative#invoking_your_service_from_the_command_line)
but `kn` seems to be more powerful.

In [3]:
# making sure kn is set up correctly
!kn service ls

NAME            URL                                            LATEST                AGE   CONDITIONS   READY   REASON
autoscale-go    http://autoscale-go.default.kn.nima-dev.com    autoscale-go-00017    25d   3 OK / 3     True    
bench1          http://bench1.default.kn.nima-dev.com          bench1-00037          34d   3 OK / 3     True    
helloworld-go   http://helloworld-go.default.kn.nima-dev.com   helloworld-go-00001   35d   3 OK / 3     True    


In [4]:
os.environ['EXP_CONFIG_NAME'] = 'bench1_sleep_rand2_1000_200'
exp_config_name = os.getenv("EXP_CONFIG_NAME")
exp_file = f"configs/{exp_config_name}.json"

workload_spec = util.load_json_file(exp_file)

kn_command = kube.get_kn_command(**workload_spec)
print(kn_command)
# to run the command, we can simply use:
# !{kn_command}

kn service apply bench1 --image ghcr.io/nimamahmoudi/conc-workloads-bench1:sha-5966a0e \
  --env EXPERIMENT_NAME=TEST1 \
  --env REPORT_INTERVAL=10 \
  --env SOCKETIO_SERVER=NO \
  --limit 'cpu=250m,memory=256Mi' \
  -a autoscaling.knative.dev/panicThresholdPercentage=1000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency


# Workload Specification

In [5]:
# get user workload function from the file
user_workload_func = request_funcs.workload_funcs[workload_spec['request_func']]
# get ready count callback
get_ready_cb = lambda: get_knative_watch_info(workload_spec['name'])['ready_replicas']
print('ready callback:', get_ready_cb())
# create logger and check one execution of workload func
wlogger = workload.WorkloadLogger(get_ready_cb=get_ready_cb)
simple_worker_func = lambda: wlogger.worker_func(user_workload_func)
# add worker func to workload spec
workload_spec['simple_worker_func'] = simple_worker_func

simple_worker_func()

ready callback: 0


{'client_start_time': 1614288108.8912876,
 'client_end_time': 1614288114.9435916,
 'client_elapsed_time': 6.052304029464722,
 'start_conc': 1,
 'end_conc': 1,
 'success': True,
 'start_ready_count': 0,
 'end_ready_count': 1}

In [6]:
# wlogger.monitoring_thread.start()
# wlogger.record_conc_loop()
# wlogger.monitor_conc_loop()
wlogger.start_capturing()
time.sleep(7)
wlogger.stop_capturing()
wlogger.get_recorded_data()

starting threads
stopping threads...
Done.


{'ready_count': [1, 1, 1, 1],
 'total_conc': [0, 0, 0, 0],
 'conc_window_average': [0.0, 0.0, 0.0, 0.0],
 'time': [1614288114.9577608,
  1614288116.9632325,
  1614288118.9718108,
  1614288120.977412]}

# Performing A Series of Experiments

In [7]:
target_list = workload_spec['exp_spec']['target_list']
rps_list = workload_spec['exp_spec']['rps_list']

In [8]:
overall_results = []
total_configs = list(itertools.product(target_list, rps_list))
for target, rps in tqdm(total_configs):
    single_res_name = perform_experiment(rps=rps, target=target, base_workload_spec=workload_spec, wlogger=wlogger)
    overall_results.append({
        'target': target,
        'rps': rps,
        'res_name': single_res_name,
        'request_func': workload_spec['request_func'],
        'workload_name': workload_spec['name'],
        'is_rps': workload_spec['is_rps'],
        'exp_time_mins': workload_spec['exp_spec']['time_mins'],
    })


  0%|          | 0/48 [00:00<?, ?it/s]

applying kn command:
kn service apply bench1 --image ghcr.io/nimamahmoudi/conc-workloads-bench1:sha-5966a0e \
  --env EXPERIMENT_NAME=TEST1 \
  --env REPORT_INTERVAL=10 \
  --env SOCKETIO_SERVER=NO \
  --limit 'cpu=250m,memory=256Mi' \
  -a autoscaling.knative.dev/panicThresholdPercentage=1000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bench1' in namespace 'default':

  0.079s The Configuration is still working to reflect the latest desired specification.
 10.506s Traffic is not yet migrated to the latest revision.
 10.567s Ingress has not yet been reconciled.
 10.665s Waiting for load balancer to be ready
 10.856s Ready to serve.

Service 'bench1' applied to latest revision 'bench1-00038' is available at URL:
http://bench1.default.kn.nima-dev.com
kn apply done
stopping threads...
Done.
starting threads
============ Experiment Started ============
Time Started: 2021-02-25 16:22:24.187195-05:00
Total Requests Made: 3552
st

In [9]:
now = get_time_with_tz()
res_name = 'overview_' + now.strftime('res-%Y-%m-%d_%H-%M-%S')
res_file_name = res_name + ".csv"
res_folder = f'results/{workload_spec["exp_spec"]["name"]}/'
df = pd.DataFrame(data=overall_results)
df.to_csv(os.path.join(res_folder, res_file_name))

df

,target,rps,res_name,request_func,workload_name,is_rps,exp_time_mins
0,1,1,res-2021-02-25_17-23-32,bench1_sleep_rand2_1000_200,bench1,False,60
1,1,2,res-2021-02-25_18-24-16,bench1_sleep_rand2_1000_200,bench1,False,60
2,1,3,res-2021-02-25_19-24-47,bench1_sleep_rand2_1000_200,bench1,False,60
3,1,5,res-2021-02-25_20-25-14,bench1_sleep_rand2_1000_200,bench1,False,60
4,1,7,res-2021-02-25_21-25-36,bench1_sleep_rand2_1000_200,bench1,False,60
5,1,10,res-2021-02-25_22-25-54,bench1_sleep_rand2_1000_200,bench1,False,60
6,1,15,res-2021-02-25_23-26-10,bench1_sleep_rand2_1000_200,bench1,False,60
7,1,20,res-2021-02-26_00-26-27,bench1_sleep_rand2_1000_200,bench1,False,60
8,2,1,res-2021-02-26_01-27-43,bench1_sleep_rand2_1000_200,bench1,False,60
9,2,2,res-2021-02-26_02-28-24,bench1_sleep_rand2_1000_200,bench1,False,60
